In [ ]:
import os
import io
import numpy as np
import glob

import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()

REMOTE_GPU = True
# This tutorial doesn't use videos, and therefore doesn't need custom kernels.
if REMOTE_GPU:
  print("here")

  from google3.learning.brain.contrib.eager.python import remote_eager
  from google3.third_party.tensorflow.python.eager import context

  # Replace with your BNS address (omit the task index)
  WORKER_NMB = '14078055'
  BNS_ADDRESS = '/bns/li/borg/li/bns/etr/etr_headless_gpu_{}.1.gpu_worker'.format(WORKER_NMB)
  NUM_WORKERS = 1

  # Connect
  print("Connecting")
  remote_eager.enable('{}/0'.format(BNS_ADDRESS), num_workers=NUM_WORKERS)
  print("Connected")

  # Print devices
  print(tf.config.experimental.list_logical_devices())

#from tensorflow.keras.layers import Dense

import matplotlib.pyplot as plt # visualization

import numpy as onp
from collections import defaultdict

import itertools
import tensorflow_datasets as tfds
import tensorflow.compat.v2 as tf
import matplotlib.pyplot as plt

import IPython.display as display
from PIL import Image
import numpy as np
import os


In [ ]:
from colabtools import adhoc_import
import getpass

client_name = 'twp'
user_name = getpass.getuser()
with adhoc_import.Google3CitcClient(
    client_name, user_name) as outputs:
  from google3.experimental.selforg.metalearn import core
  core = adhoc_import.Reload(core)
  from google3.experimental.selforg.metalearn import tf_layers
  tf_layers = adhoc_import.Reload(tf_layers)
  from google3.experimental.selforg.metalearn import util
  util = adhoc_import.Reload(util)
  from google3.experimental.selforg.neurca.data import data_processing
  data_processing = adhoc_import.Reload(data_processing)
  from google3.experimental.selforg.metalearn import training
  training = adhoc_import.Reload(training)

  #import umap
  #from google3.third_party.py.umap import umap_
  #umap_ = adhoc_import.Reload(umap_)

MPDense = tf_layers.MPDense
#MPReshape = layers.MPReshape
#MPConv = tf_layers.MPConv
#MPWeightNorm = layers.MPWeightNorm
MPActivation = tf_layers.MPActivation
MPSoftmax = tf_layers.MPSoftmax
#MPStandardize = layers.MPStandardize
MPCrossEntropyLoss = tf_layers.MPCrossEntropyLoss
MPNetwork = tf_layers.MPNetwork
MPMetrics = tf_layers.MPMetrics
SamplePool = util.SamplePool
load_benchmark_MNIST = data_processing.load_benchmark_MNIST
TrainingRegime = training.TrainingRegime

GRUBlock = core.GRUBlock
OutStandardizer = core.OutStandardizer
StandardizeInputsAndStates = core.StandardizeInputsAndStates


In [ ]:
def one_hottify(dset):
  one_hottified = onp.zeros([dset.shape[0], 10], dtype=onp.float32)
  one_hottified[onp.arange(dset.size), dset] = 1.0
  return one_hottified

def MNIST_generator(inputs, labels):
  while True:
    idx = pyrandom.randrange(len(inputs))
    x = inputs[idx]
    y = labels[idx]
    yield x, y

PIC_L = 12

(x_train, y_train), (x_test, y_test) = load_benchmark_MNIST()
x_train = tf.image.resize(onp.expand_dims(x_train, -1), size=(PIC_L, PIC_L)).numpy()
x_test = tf.image.resize(onp.expand_dims(x_test, -1), size=(PIC_L, PIC_L)).numpy()
y_train = one_hottify(y_train)
y_test = one_hottify(y_test)

# standardize inputs:
train_mean, train_std = x_train.mean(), x_train.std()

x_train = (x_train - train_mean) / train_std
x_test = (x_test - train_mean) / train_std

x_train = x_train.reshape([-1, PIC_L * PIC_L])
x_test = x_test.reshape([-1, PIC_L * PIC_L])



In [ ]:
outer_batch_size = 1
inner_batch_size = 8
loop_steps = 20
img_h = img_w = 12

def resize(x, y):
  x = tf.image.resize(x, [img_h, img_w])
  x = tf.reshape(x, [-1, img_h * img_w])
  return x, y

resized_mnist_train = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train))#.map(resize)

with tf.device("/cpu:0"):
  dataset = resized_mnist_train.batch(inner_batch_size).batch(loop_steps).batch(outer_batch_size).shuffle(1000).cache().repeat()
  ds_iter = iter(dataset)
  xval_ds = resized_mnist_train.batch(inner_batch_size).batch(outer_batch_size).shuffle(1000).cache().repeat()
  xval_ds_iter = iter(xval_ds)

  test_ds = tf.data.Dataset.from_generator(
            lambda: MNIST_generator(x_test, y_test),
            output_types=(onp.float32, onp.float32))
  test_ds = test_ds.batch(inner_batch_size)
  test_ds_iter = iter(test_ds)


In [ ]:
message_size = 4
stateful = True
stateful_hidden_n = 7#15

MPMetrics.reset()
import functools

l0 = PIC_L * PIC_L
def init_network(activation, sizes):
  network = MPNetwork([MPDense(sizes[1], stateful=stateful, stateful_hidden_n=stateful_hidden_n),
                    MPActivation(activation, stateful=stateful, stateful_hidden_n=stateful_hidden_n),
                    MPDense(10, stateful=stateful, stateful_hidden_n=stateful_hidden_n),
                    MPSoftmax(stateful=stateful, stateful_hidden_n=stateful_hidden_n),
                    ],
                    MPCrossEntropyLoss(message_size, stateful=stateful, stateful_hidden_n=stateful_hidden_n))
  # create shared networks!
  shared_params = {
      "W_net": GRUBlock(x_dim=2 + message_size,
                        carry_n=1+message_size+stateful_hidden_n),
      "b_net": GRUBlock(x_dim=2 + message_size,
                        carry_n=1+message_size+stateful_hidden_n),
#      "W_out_std": OutStandardizer(scale_init_val=0.05),
#      "b_out_std": OutStandardizer(scale_init_val=0.05),
#      "W_in_std": StandardizeInputsAndStates(["W_b", "W_in"]),
#      "b_in_std": StandardizeInputsAndStates(["b_b", "b_in"]),
                   }

  network.setup(in_dim=sizes[0], message_size=message_size, 
                inner_batch_size=inner_batch_size,
                shared_params=shared_params)
  return network

network = init_network(tf.sigmoid, (l0, 50))


In [ ]:
# load from drive
from colabtools import drive
#from google.colab import drive
drive.ListFolder("16EFaQPhjfj4oYEYzJ_e6DCqmfP_mVVSW")#/My%20Drive")#/selforg/Learn\ 2\ Learn/mnist")

weights_byte = drive.LoadFile(file_id="13sU-0KGLPFHsDZS2mOB1o7tF-yHIOF9t")
print(weights_byte)
!ls -lh tmp
with open("tmp/weights_00005000.npy", "wb") as fout:
  fout.write(weights_byte)
  #arr = np.fromstring(weights_byte)
  #np.save(fout, arr)
!ls -lh tmp

In [ ]:
!ls -lh tmp
network.load_weights("tmp/weights")

In [ ]:
def create_new_p_fw():
  return network.init(inner_batch_size)

POOL_SIZE = 16

pool = SamplePool(ps=tf.stack(
    [network.serialize_pfw(create_new_p_fw()) for _ in range(POOL_SIZE)]).numpy())

In [ ]:
loop_steps_t = tf.constant(loop_steps)

learning_schedule = 5e-4

# Prepare a training regime.
# The heldout_weight tells you how to split the loss between train and eval sets
# that are passed to the network.
# Empirically, a heldout_weight=0.0 (or None), results in a much lower overall
# performance, both for train and test losses.
training_regime = TrainingRegime(
    network, heldout_weight=1.0, hint_loss_ratio=0.7, remember_loss_ratio=None)

last_step = 0

# Initialize important parameters by passing a minibatch.
#x_b, y_b, _, _ = next(ds_iter)
#network.minibatch_init(x_b[0][0],  y_b[0][0], INNER_BATCH_SIZE)

# enhanced version of minibatch init, allowing to initialize by looking at more
# than just one step.
# Likewise, this can be run multiple times to improve the initialization.
for j in range(5):
  print("on", j)
  stats = []
  pfw = network.init(inner_batch_size)

  x_b, y_b = next(ds_iter)
  x_b, y_b = x_b[0], y_b[0]
  for i in range(loop_steps):
    # Initialize important parameters by passing a minibatch.
    #x_b, y_b = x_eval, target_eval # a minibatch, basically. Could be prettified.

    pfw, stats_i = network.minibatch_init(x_b[i],  y_b[i], x_b[i].shape[0], pfw=pfw)
    stats.append(stats_i)
  # update
  network.update_statistics(stats, update_perc=0.5)

  print("final mean:")
  for p in tf.nest.flatten(pfw):
    print(p.shape, tf.reduce_mean(p), tf.math.reduce_std(p))



# The outer loop here uses Adam. SGD/Momentum are more stable but way slower.
#trainer = tf.keras.optimizers.Adam(learning_schedule)
trainer = tf.keras.optimizers.Adam(learning_schedule)

loss_log = []
def smoothen(l, lookback=20):
  # first of all, if it's a nan, change it to a high value
  kernel = [1./lookback] * lookback
  return np.convolve(l[0:1] * (lookback - 1) + l, kernel, "valid")


In [ ]:
!mkdir tmp

In [ ]:

training_steps = 100000

#steps_scheduler = tf.keras.optimizers.schedules.PiecewiseConstantDecay([500, 1000, 1500, 2000, 5000], [2, 10, 15, 20, 30, 50])
#steps_scheduler = tf.keras.optimizers.schedules.PiecewiseConstantDecay([5000], [50, 50])

learning_schedule = 1e-4#optimizers.piecewise_constant([1000, 2000], [1e-5, 1e-4, 1e-3])


@tf.function
def step(pfws, xts, yts, xes, yes, num_steps):
  print("compiling")
  with tf.GradientTape() as g:
    # As you can see, we pass train and eval/heldout instances.
    # We only show the network the xts and yts for the inner update,
    # but we compute a cross-validation loss using also xes and yes.
    l, _, _ = training_regime.batch_mp_loss(pfws, xts, yts, xes, yes, num_steps)
  all_weights = network.get_trainable_weights()
  grads = g.gradient(l, all_weights)
  # Try grad clipping to avoid explosions.
  grads = [g/(tf.norm(g)+1e-8) for g in grads]
  trainer.apply_gradients(zip(grads, all_weights))
  return l


import time
start_time = time.time()

#MPMetrics.reset_metrics()

for i in range(last_step + 1, last_step +1 + training_steps):
  last_step = i

  #num_loops = loop_steps # steps_scheduler(last_step)

  tmp_t = time.time()
  xts, yts = next(ds_iter)
  xes, yes = next(xval_ds_iter)

  batch = pool.sample(outer_batch_size)
  fwps = batch.ps

  l = step(fwps, xts, yts, xes, yes, loop_steps_t)
  loss_log.append(l)
  # Metrics aggregation is slow. Consider doing once in a while to now interrupt
  #  all the time
  #add_metric_episode(metrics_series, merge_metric_batch(metrics))


  if i % 50 == 0:
    print(i)
    print("--- %s seconds ---" % (time.time() - start_time))
  if i % 500 == 0:
    plt.plot(smoothen(loss_log, 100), label='mp')
    plt.yscale('log')
    #plt.ylim(0.0, 1e-1)
    plt.legend()
    plt.show()
  if i % 2500 == 0:
    file_path = "tmp/weights"
    network.save_weights(file_path, last_step)
print("--- %s seconds ---" % (time.time() - start_time))

np_batched_stl_mpv2_nograd_loss = time_series



#print("After")
#print_dicts(p_fw)


In [ ]:
!ls tmp -lh

In [ ]:
def smoothen(l, lookback=20):
  kernel = [1./lookback] * lookback
  return onp.convolve(l[0:1] * (lookback - 1) + l, kernel, "valid")

plt.plot(smoothen(loss_log, 100), label='mp')
plt.yscale('log')
plt.ylim(1e-2, 3e-1)
plt.legend()
plt.show()

In [ ]:
# save the last model
! ls tmp -lh
%download_file tmp/weights_00005000.npy

In [ ]:
# @title Optionally, load from drive.
# load the proper file.
loaded_weights = %upload_files
print(loaded_weights)
!mkdir tmp
f = open('tmp/weights_00005000.npy', 'wb')
f.write(loaded_weights['weights_00005000.npy'])
f.close()
a = np.load('tmp/weights_00005000.npy')
print(a.shape)

In [ ]:
#@title if you loaded the file, reload the network

network = init_network(tf.sigmoid, (l0, 50))
network.load_weights("tmp/weights")



In [ ]:
# Now let's generate a training regime of 10 steps, and compare with SGD.

eval_pfw = create_new_p_fw()

def prepare_for_analysis(pfw):
  return tf.concat([tf.reshape(t, [-1]) for t in pfw], 0).numpy()


print(prepare_for_analysis(eval_pfw).shape)

In [ ]:

test_sp_bs = 100

with tf.device("/cpu:0"):
  dataset_sp = tf.data.Dataset.from_tensor_slices((x_train, y_train))
  dataset_sp = dataset_sp.batch(inner_batch_size).shuffle(1000).repeat()
  dataset_sp_iter = iter(dataset_sp)
  test_sp_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
  test_sp_ds = test_sp_ds.batch(test_sp_bs).repeat()
  test_sp_ds_iter = iter(test_sp_ds)


def get_accuracy(pfw, xe, ye):
  targets = tf.argmax(ye, axis=-1)
  res, _, _ = network.forward(pfw, xe)
  predictions = tf.argmax(res, axis=-1)

  tot_correct = tf.reduce_sum(tf.cast(tf.equal(predictions, targets), tf.float32))
  accuracy = tot_correct / ye.shape[0]

  return accuracy

In [ ]:
# MP step accuracy
all_MP_series = []
eval_tot_steps = 100
ev_losses = np.zeros([eval_tot_steps, loop_steps])
for s in range(eval_tot_steps):
  if s % 10 == 0:
    print("\nRepetition {}".format(s))
  MP_params_series = []

  mp_pfw = network.init(inner_batch_size)

  #MP_params_series.append(prepare_for_analysis(mp_pfw))
  #MP_params_series.append(mp_pfw)

  for i in range(loop_steps):
    # using test to train too, because we want to see the effect of learning
    # on the parameter space.
    xt, yt = next(dataset_sp_iter)
    
    mp_pfw, _, _ = network.inner_update(mp_pfw, xt, yt)
    #MP_params_series.append(prepare_for_analysis(mp_pfw))
    #MP_params_series.append(mp_pfw)

    xe, ye = next(test_sp_ds_iter)
    accuracy = get_accuracy(mp_pfw, xe, ye)
    ev_losses[s, i] = accuracy

    #print("step {}: accuracy {}". format(i, float(accuracy)))
ev_losses_m = np.mean(ev_losses, axis=0)
ev_losses_sd = np.std(ev_losses, axis=0)
print("mean:", ev_losses_m)
print("sd:", ev_losses_sd)

ub = [m + sd for m, sd in zip(ev_losses_m, ev_losses_sd)]
lb = [m - sd for m, sd in zip(ev_losses_m, ev_losses_sd)]
plt.fill_between(range(1, len(ev_losses_m) + 1), ub, lb, alpha=.5)
plt.plot(range(1, len(ev_losses_m) + 1), ev_losses_m, label='eval loss')
#plt.ylim(0.0, 0.04)
plt.xlabel("num steps")
plt.ylabel("Accuracy")
plt.legend()

  #all_MP_series.append(MP_params_series)

mp_baseline_m = ev_losses_m

In [ ]:
# MP step
all_MP_series = []
for s in range(10):
  print("\nRepetition {}".format(s))
  MP_params_series = []

  mp_pfw = network.init(inner_batch_size)

  #MP_params_series.append(prepare_for_analysis(mp_pfw))
  #MP_params_series.append(mp_pfw)

  for i in range(10):
    # using test to train too, because we want to see the effect of learning
    # on the parameter space.
    xt, yt = next(dataset_sp_iter)
    
    mp_pfw, _, _ = inner_update(mp_pfw, xt, yt)
    #MP_params_series.append(prepare_for_analysis(mp_pfw))
    #MP_params_series.append(mp_pfw)

    xe, ye = next(test_sp_ds_iter)
    accuracy = get_accuracy(mp_pfw, xe, ye)
    print("step {}: accuracy {}". format(i, float(accuracy)))


  #all_MP_series.append(MP_params_series)


In [ ]:
# ADAM network
adam_network = MPNetwork([MPDense(50, stateful=False),
                  MPActivation(tf.sigmoid, stateful=False),
                  MPDense(10, stateful=False),
                  MPSoftmax(stateful=False),
                  ],
                  MPCrossEntropyLoss(message_size, stateful=False))
adam_network.setup(in_dim=l0, message_size=message_size)



def get_adam_accuracy(pfw, xe, ye):
  targets = tf.argmax(ye, axis=-1)
  res, _, _ = adam_network.forward(pfw, xe)
  predictions = tf.argmax(res, axis=-1)

  tot_correct = tf.reduce_sum(tf.cast(tf.equal(predictions, targets), tf.float32))
  accuracy = tot_correct / ye.shape[0]

  return accuracy

In [ ]:
# SGD step



all_SGD_series = []
eval_tot_steps = 100
sgd_ev_losses = np.zeros([eval_tot_steps, loop_steps])
for s in range(eval_tot_steps):
  if s % 10 == 0:
    print("\nRepetition {}".format(s))
  #print("on", s)
  SGD_params_series = []

  sgd_pfw = [tf.Variable(t) for t in adam_network.init()]

  #SGD_params_series.append(prepare_for_analysis(sgd_pfw))
  #SGD_params_series.append(sgd_pfw)

  adam_trainer = tf.keras.optimizers.SGD(0.1)

  @tf.function
  def step(xt, yt):
    with tf.GradientTape() as g:
      g.watch(sgd_pfw)
      y, _, _ = adam_network.forward(sgd_pfw, xt)
      l, _ = adam_network.compute_loss(y, yt)
      l = tf.reduce_mean(tf.reduce_sum(l, axis=[1]))

    grads = g.gradient(l, sgd_pfw)
    adam_trainer.apply_gradients(zip(grads, sgd_pfw))


  for i in range(loop_steps):
    # using test to train too, because we want to see the effect of learning
    # on the parameter space.
    xt, yt = next(dataset_sp_iter)
    step(xt, yt)

    if i % 1 == 0:
      xe, ye = next(test_sp_ds_iter)
      accuracy = get_adam_accuracy(sgd_pfw, xe, ye)
      sgd_ev_losses[s, i] = accuracy
      #print("step {}: accuracy {}". format(i, float(accuracy)))
      #SGD_params_series.append(sgd_pfw)
      #SGD_params_series.append(prepare_for_analysis(sgd_pfw))
  #all_SGD_series.append(SGD_params_series)

ev_losses_m = np.mean(sgd_ev_losses, axis=0)
ev_losses_sd = np.std(sgd_ev_losses, axis=0)
print("mean:", ev_losses_m)
print("sd:", ev_losses_sd)

ub = [m + sd for m, sd in zip(ev_losses_m, ev_losses_sd)]
lb = [m - sd for m, sd in zip(ev_losses_m, ev_losses_sd)]
plt.fill_between(range(1, len(ev_losses_m) + 1), ub, lb, alpha=.5)
plt.plot(range(1, len(ev_losses_m) + 1), ev_losses_m, label='eval loss')
#plt.ylim(0.0, 0.04)
plt.xlabel("num steps")
plt.ylabel("Accuracy")
plt.legend()

sgd_m = ev_losses_m

In [ ]:
# Adam step



all_SGD_series = []
eval_tot_steps = 100
sgd_ev_losses = np.zeros([eval_tot_steps, loop_steps])
for s in range(eval_tot_steps):
  if s % 10 == 0:
    print("\nRepetition {}".format(s))
  #print("on", s)
  SGD_params_series = []

  sgd_pfw = [tf.Variable(t) for t in adam_network.init()]

  #SGD_params_series.append(prepare_for_analysis(sgd_pfw))
  #SGD_params_series.append(sgd_pfw)

  adam_trainer = tf.keras.optimizers.Adam(0.01)

  @tf.function
  def step(xt, yt):
    with tf.GradientTape() as g:
      g.watch(sgd_pfw)
      y, _, _ = adam_network.forward(sgd_pfw, xt)
      l, _ = adam_network.compute_loss(y, yt)
      l = tf.reduce_mean(tf.reduce_sum(l, axis=[1]))

    grads = g.gradient(l, sgd_pfw)
    adam_trainer.apply_gradients(zip(grads, sgd_pfw))


  for i in range(loop_steps):
    # using test to train too, because we want to see the effect of learning
    # on the parameter space.
    xt, yt = next(dataset_sp_iter)
    step(xt, yt)

    if i % 1 == 0:
      xe, ye = next(test_sp_ds_iter)
      accuracy = get_adam_accuracy(sgd_pfw, xe, ye)
      sgd_ev_losses[s, i] = accuracy
      #print("step {}: accuracy {}". format(i, float(accuracy)))
      #SGD_params_series.append(sgd_pfw)
      #SGD_params_series.append(prepare_for_analysis(sgd_pfw))
  #all_SGD_series.append(SGD_params_series)

ev_losses_m = np.mean(sgd_ev_losses, axis=0)
ev_losses_sd = np.std(sgd_ev_losses, axis=0)
print("mean:", ev_losses_m)
print("sd:", ev_losses_sd)

ub = [m + sd for m, sd in zip(ev_losses_m, ev_losses_sd)]
lb = [m - sd for m, sd in zip(ev_losses_m, ev_losses_sd)]
plt.fill_between(range(1, len(ev_losses_m) + 1), ub, lb, alpha=.5)
plt.plot(range(1, len(ev_losses_m) + 1), ev_losses_m, label='eval loss')
#plt.ylim(0.0, 0.04)
plt.xlabel("num steps")
plt.ylabel("Accuracy")
plt.legend()

adam_m = ev_losses_m

In [ ]:
raise Exception("Don't run code below! it'd destroy your model.")

In [ ]:
# ADAM step

all_ADAM_series = []
for s in range(10):
  print("on", s)
  ADAM_params_series = []

  adam_pfw = [tf.Variable(t) for t in eval_pfw]

  ADAM_params_series.append(prepare_for_analysis(adam_pfw))
  #ADAM_params_series.append(adam_pfw)

  trainer = tf.keras.optimizers.Adam(0.01)


  for i in range(10):
    # using test to train too, because we want to see the effect of learning
    # on the parameter space.
    xt, yt = next(test_ds_iter)
    targets = tf.argmax(yt, axis=-1)

    with tf.GradientTape() as g:
      g.watch(adam_pfw)
      y, _, _ = network.forward(adam_pfw, xt)
      l, _ = network.compute_loss(y, yt)
      l = tf.reduce_mean(tf.reduce_sum(l, axis=[1]))

    grads = g.gradient(l, adam_pfw)
    trainer.apply_gradients(zip(grads, adam_pfw))

    if i % 1 == 0:
      accuracy = get_accuracy(adam_pfw, xe, ye)
      print("step {}: accuracy {}". format(i, float(accuracy)))
      ADAM_params_series.append(prepare_for_analysis(adam_pfw))
      #ADAM_params_series.append(adam_pfw)

  all_ADAM_series.append(ADAM_params_series)


In [ ]:
# MP step but starting from a new pfw

new_eval_pfw = create_new_p_fw()

all_MP_series_v2 = []
for s in range(10):
  print("\nRepetition {}".format(s))
  MP_params_series = []

  mp_pfw = new_eval_pfw

  MP_params_series.append(prepare_for_analysis(mp_pfw))
  #MP_params_series.append(mp_pfw)

  for i in range(10):
    # using test to train too, because we want to see the effect of learning
    # on the parameter space.
    xt, yt = next(dataset_sp_iter)
    
    mp_pfw, _, _ = inner_update(mp_pfw, xt, yt)
    MP_params_series.append(prepare_for_analysis(mp_pfw))
    #MP_params_series.append(mp_pfw)

    xe, ye = next(test_sp_ds_iter)
    accuracy = get_accuracy(mp_pfw, xe, ye)
    print("step {}: accuracy {}". format(i, float(accuracy)))


  all_MP_series_v2.append(MP_params_series)


In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import pandas as pd


In [ ]:
all_inputs = []
labels = []

add_iter = False

for idx, s in enumerate(all_MP_series):

  all_inputs += s
  curr_lbl = "MP"
  curr_lbl += "_{}".format(idx) if add_iter else ""
  labels += [curr_lbl for _ in range(len(s))]

for idx, s in enumerate(all_SGD_series):
  all_inputs += s
  curr_lbl = "SGD_1e-1"
  curr_lbl += "_{}".format(idx) if add_iter else ""  
  labels += [curr_lbl for _ in range(len(s))]

for idx, s in enumerate(all_ADAM_series):
  all_inputs += s
  curr_lbl = "ADAM_1e-2"
  curr_lbl += "_{}".format(idx) if add_iter else ""  
  labels += [curr_lbl for _ in range(len(s))]

for idx, s in enumerate(all_MP_series_v2):
  all_inputs += s
  curr_lbl = "MP_v2"
  curr_lbl += "_{}".format(idx) if add_iter else ""
  labels += [curr_lbl for _ in range(len(s))]

X = np.stack(all_inputs)

print(X.shape)

In [ ]:
pca = PCA(n_components=2)
pca_result = pca.fit_transform(X)


In [ ]:
# ["MP_{}".format(i) for i in range(X.shape[0])]
df = pd.DataFrame({"label": labels,
                   "pca-one": pca_result[:, 0], "pca-two": pca_result[:,1]})


In [ ]:
import seaborn as sns
plt.figure(figsize=(16,10))

sns.scatterplot(
    x="pca-one", y="pca-two",
    hue="label",
    #palette=sns.color_palette("hls", 11),
    data=df,
    legend="full",
    alpha=0.8
)

#sns.lineplot(x="pca-one", y="pca-two", hue="label", data=df)

In [ ]:
import scipy
MP_inst = all_MP_series[0]
MP_inst_v2 = all_MP_series_v2[0]
mp_origin = MP_inst[0]
sgd_inst = all_SGD_series[0]
adam_inst = all_ADAM_series[0]
for i in range(1, len(MP_inst)):
  mp_next = MP_inst[i] - mp_origin
  mp_v2_next = MP_inst_v2[i] - mp_origin
  sgd_next = sgd_inst[i] - mp_origin
  adam_next = adam_inst[i] - mp_origin
  print(scipy.spatial.distance.cosine(mp_origin, mp_next),
        scipy.spatial.distance.cosine(mp_origin, sgd_next),
        scipy.spatial.distance.cosine(mp_origin, adam_next),
        scipy.spatial.distance.cosine(mp_next, sgd_next),
        scipy.spatial.distance.cosine(mp_next, adam_next),
        scipy.spatial.distance.cosine(sgd_next, adam_next),
        scipy.spatial.distance.cosine(mp_next, mp_v2_next))

# See what happens if you change activation

In [ ]:
#message_size = 4#8
#stateful = True
#stateful_hidden_n = 7#15


new_activation = lambda x: tf.maximum(0.0, tf.sign(x))

l0 = PIC_L * PIC_L
"""
def init_new_network():
  new_network = MPNetwork([MPDense(50, stateful=stateful, stateful_hidden_n=stateful_hidden_n),
                    MPActivation(new_activation, stateful=stateful, stateful_hidden_n=stateful_hidden_n),
                    MPDense(10, stateful=stateful, stateful_hidden_n=stateful_hidden_n),
                    MPSoftmax(stateful=stateful, stateful_hidden_n=stateful_hidden_n),
                    ],
                    MPCrossEntropyLoss(message_size, stateful=stateful, stateful_hidden_n=stateful_hidden_n))
  new_network.setup(in_dim=l0, message_size=message_size, inner_batch_size=inner_batch_size)
  return new_network"""

new_network = init_network(new_activation, (l0, 50))
new_network.load_weights("tmp/weights")

def get_accuracy(net, pfw, xe, ye):
  targets = tf.argmax(ye, axis=-1)
  res, _, _ = net.forward(pfw, xe)
  predictions = tf.argmax(res, axis=-1)

  tot_correct = tf.reduce_sum(tf.cast(tf.equal(predictions, targets), tf.float32))
  accuracy = tot_correct / ye.shape[0]

  return accuracy

In [ ]:
# MP step accuracy
all_MP_series = []
eval_tot_steps = 100
ev_losses = np.zeros([eval_tot_steps, loop_steps])
for s in range(eval_tot_steps):
  if s % 10 == 0:
    print("\nRepetition {}".format(s))
  MP_params_series = []

  mp_pfw = new_network.init(inner_batch_size)

  #MP_params_series.append(prepare_for_analysis(mp_pfw))
  #MP_params_series.append(mp_pfw)

  for i in range(loop_steps):
    # using test to train too, because we want to see the effect of learning
    # on the parameter space.
    xt, yt = next(dataset_sp_iter)
    
    mp_pfw, _, _ = new_network.inner_update(mp_pfw, xt, yt)
    #MP_params_series.append(prepare_for_analysis(mp_pfw))
    #MP_params_series.append(mp_pfw)

    xe, ye = next(test_sp_ds_iter)
    accuracy = get_accuracy(new_network, mp_pfw, xe, ye)
    ev_losses[s, i] = accuracy

    #print("step {}: accuracy {}". format(i, float(accuracy)))
ev_losses_m = np.mean(ev_losses, axis=0)
ev_losses_sd = np.std(ev_losses, axis=0)
print("mean:", ev_losses_m)
print("sd:", ev_losses_sd)

ub = [m + sd for m, sd in zip(ev_losses_m, ev_losses_sd)]
lb = [m - sd for m, sd in zip(ev_losses_m, ev_losses_sd)]
plt.fill_between(range(1, len(ev_losses_m) + 1), ub, lb, alpha=.5)
plt.plot(range(1, len(ev_losses_m) + 1), ev_losses_m, label='eval loss')
#plt.ylim(0.0, 0.04)
plt.xlabel("num steps")
plt.ylabel("Accuracy")
plt.legend()

  #all_MP_series.append(MP_params_series)
mp_stepf_m = ev_losses_m

# See what happens if you transfer the learned parameters

In [ ]:
# big version of MNIST
(x_train_b, y_train_b), (x_test_b, y_test_b) = load_benchmark_MNIST()
y_train_b = one_hottify(y_train_b)
y_test_b = one_hottify(y_test_b)

# standardize inputs:
train_mean, train_std = x_train_b.mean(), x_train_b.std()

x_train_b = (x_train_b - train_mean) / train_std
x_test_b = (x_test_b - train_mean) / train_std

x_train_b = x_train_b.reshape([-1, 28 * 28])
x_test_b = x_test_b.reshape([-1, 28* 28])


In [ ]:
#message_size = 4#8
#stateful = True
#stateful_hidden_n = 7#15

MPMetrics.reset()
import functools
"""
def init_big_network():
  new_network = MPNetwork([MPDense(100, stateful=stateful, stateful_hidden_n=stateful_hidden_n),
                    MPActivation(tf.sigmoid, stateful=stateful, stateful_hidden_n=stateful_hidden_n),
                    MPDense(10, stateful=stateful, stateful_hidden_n=stateful_hidden_n),
                    MPSoftmax(stateful=stateful, stateful_hidden_n=stateful_hidden_n),
                    ],
                    MPCrossEntropyLoss(message_size, stateful=stateful, stateful_hidden_n=stateful_hidden_n))
  new_network.setup(in_dim=28 * 28, message_size=message_size, inner_batch_size=inner_batch_size)
  return new_network"""

new_network = init_network(tf.sigmoid, (28*28, 100))
new_network.load_weights("tmp/weights")


In [ ]:
!ls tmp

In [ ]:

test_sp_bs = 100

with tf.device("/cpu:0"):
  dataset_sp_b = tf.data.Dataset.from_tensor_slices((x_train_b, y_train_b))
  dataset_sp_b = dataset_sp_b.batch(inner_batch_size).shuffle(1000).repeat()
  dataset_sp_b_iter = iter(dataset_sp_b)
  test_sp_b_ds = tf.data.Dataset.from_tensor_slices((x_test_b, y_test_b))
  test_sp_b_ds = test_sp_b_ds.batch(test_sp_bs).repeat()
  test_sp_b_ds_iter = iter(test_sp_b_ds)


def get_accuracy(pfw, xe, ye):
  targets = tf.argmax(ye, axis=-1)
  res, _, _ = new_network.forward(pfw, xe)
  predictions = tf.argmax(res, axis=-1)

  tot_correct = tf.reduce_sum(tf.cast(tf.equal(predictions, targets), tf.float32))
  accuracy = tot_correct / ye.shape[0]

  return accuracy

In [ ]:
# MP step accuracy
all_MP_series = []
eval_tot_steps = 100
ev_losses = np.zeros([eval_tot_steps, loop_steps])
for s in range(eval_tot_steps):
  if s % 10 == 0:
    print("\nRepetition {}".format(s))
  MP_params_series = []

  mp_pfw = new_network.init(inner_batch_size)

  #MP_params_series.append(prepare_for_analysis(mp_pfw))
  #MP_params_series.append(mp_pfw)

  for i in range(loop_steps):
    # using test to train too, because we want to see the effect of learning
    # on the parameter space.
    xt, yt = next(dataset_sp_b_iter)
    
    mp_pfw, _, _ = new_network.inner_update(mp_pfw, xt, yt)
    #MP_params_series.append(prepare_for_analysis(mp_pfw))
    #MP_params_series.append(mp_pfw)

    xe, ye = next(test_sp_b_ds_iter)
    accuracy = get_accuracy(mp_pfw, xe, ye)
    ev_losses[s, i] = accuracy

    #print("step {}: accuracy {}". format(i, float(accuracy)))
ev_losses_m = np.mean(ev_losses, axis=0)
ev_losses_sd = np.std(ev_losses, axis=0)
print("mean:", ev_losses_m)
print("sd:", ev_losses_sd)

ub = [m + sd for m, sd in zip(ev_losses_m, ev_losses_sd)]
lb = [m - sd for m, sd in zip(ev_losses_m, ev_losses_sd)]
plt.fill_between(range(1, len(ev_losses_m) + 1), ub, lb, alpha=.5)
plt.plot(range(1, len(ev_losses_m) + 1), ev_losses_m, label='eval loss')
#plt.ylim(0.0, 0.04)
plt.xlabel("num steps")
plt.ylabel("Accuracy")
plt.legend()

  #all_MP_series.append(MP_params_series)
mp_big_net_m = ev_losses_m

In [ ]:
# plot all together.

x_values = range(1, len(mp_baseline_m) + 1)

plt.plot(x_values, mp_baseline_m * 100, label='MPLP trained')
plt.plot(x_values, mp_stepf_m * 100, label='MPLP step function')
plt.plot(x_values, mp_big_net_m * 100, label='MPLP on bigger network')
plt.plot(x_values, sgd_m * 100, label='SGD')
plt.plot(x_values, adam_m * 100, label='Adam')
plt.xticks([4, 8, 12, 16, 20])

#plt.ylim(0.0, 0.04)
plt.xlabel("Number of steps")
plt.ylabel("Accuracy (%)")
plt.legend()


with open("tmp/mplp_evals.png", "wb") as fout:
  plt.savefig(fout)
%download_file tmp/mplp_evals.png